In [1]:
import csv
import pandas as pd
import sqlite3
import numpy as np
import os
import sys
import math

In [2]:
database = '../fia.sqlite'
conn = sqlite3.connect(database)

In [4]:
sql = "SELECT plott.invyr, plott.statecd, plott.unitcd, plott.countycd, plott.plot, lat, lon, AVG(slope), AVG(aspect), AVG(elev) FROM forest_inventory_analysis_COND as cond JOIN forest_inventory_analysis_PLOT as plott ON cond.statecd == plott.statecd AND cond.unitcd == plott.unitcd AND cond.countycd == plott.countycd AND cond.plot == plott.plot WHERE slope != '' AND elev != '' AND aspect != '' GROUP BY plott.invyr, plott.statecd, plott.unitcd, plott.countycd, plott.plot"
fia_climate = pd.read_sql_query(sql, conn)

In [6]:
#Getting all predictor values
Neon_Domain3 = pd.read_csv("../data/domain3.csv")
bioclim = pd.read_csv("../data/bioclim_fia.csv")
bioclim.drop('Unnamed: 0', axis=1, inplace=True)

FileNotFoundError: File b'../data/domain3.csv' does not exist

In [5]:
sql = "SELECT COUNT(*) ,spcd, statecd, unitcd, countycd, plot, invyr FROM forest_inventory_analysis_TREE GROUP BY spcd, statecd, unitcd, countycd, plot, invyr"
fia_response = pd.read_sql_query(sql, conn)

In [6]:
conn.close()

In [7]:
'''
@param 
df: the left data frame
df2: the right data frame
merge: the colunms to merge the data frames
join_kind: type of join on the two data frames (left, inner, right, outer)
@return 
final_df: a data frame that is a combination of the two data frames
'''
def format_predictors(df, df2=None, merge=["statecd","unitcd","countycd","plot"], join_kind="left"):
    df.columns = map(str.lower, df.columns)
    if(df2.equals(None)):
        #merging all of the duplicate rows
        final_df = df.groupby(merge, as_index=False).mean()
    else:
        #renaming columns to lowercase
        df2.columns = map(str.lower, df2.columns)
        final_df = pd.merge(df, df2, on=merge, how=join_kind)
        final_df = final_df.groupby(merge, as_index=False).mean()
        #adding primary key for the plots
        final_df['id_coords'] = final_df.apply(lambda row: str(int(row.statecd)) + '_' + str(int(row.unitcd)) + '_' + str(int(row.countycd)) + '_' + str(int(row[3])), axis=1)
        #for duplicate lat and lon in the data frames dropping one of them and renaming the other
    if "lat_x" in final_df.columns:
        final_df.drop(["lat_y", "lon_y"], axis=1, inplace=True)
        final_df.rename(columns={"lat_x":"lat","lon_x":"lon"},inplace=True)
    return final_df

In [8]:
neon_domain3 = format_predictors(Neon_Domain3, fia_climate)
neon_climate = format_predictors(neon_domain3, bioclim)
cont_climate = format_predictors(fia_climate,bioclim)

In [253]:
def kmvar(df):
    module_path = os.path.abspath(os.path.join('../Same-Size-K-Means/'))
    if module_path not in sys.path:
        sys.path.append(module_path)
    from clustering.equal_groups import EqualGroupsKMeans
    clf = EqualGroupsKMeans(n_clusters=int(math.ceil(df.shape[0]/16)), random_state=0)
    clf.fit(df)
    df['labels'] = clf.labels_
    return np.array(df['labels'])

In [254]:
'''
@param
df: the data frame of the predictors
cols_to_cluster: columns to cluster the data frame by
group_by: columns to split the data frames by
predictors: the column name of the predictors to average in the clusters
id_coord_dict: a dictionary that stores the key: id_coords and value: row number in the data frame
@return
A data frame that has the clustered plots as a single row
'''    
    
def cluster_plots(df, cols_to_cluster, id_coord_dict, group_by=["statecd","unitcd"]):
    clustered_df = pd.DataFrame(columns=df.columns[:-1])
    id_list = df['id_coords']
    grouped_df = [x for x in df.loc[:, df.columns!='id_coords'].groupby(group_by)]
    r = 0
    for group in grouped_df:
        temp = group[1].loc[:,['lat','lon']]
        clust_group = kmvar(temp)
        #largest number in the cluster identifiers is the number of clusters
        num = max(clust_group)
        print(clust_group)
        for i in range(num):
            ii = np.where(clust_group == i)
            print(ii[0])
            if(ii[0].size == 16):
                add_df = group[1].loc[ii[0]].mean(axis=0)
                clustered_df.loc[r, clustered_df.columns] =add_df
                r += 1
                #adding all id coords as key and the row number in the clustered_df as value to the dictionary
                for ids in id_list[ii[0]]:
                    id_coord_dict[ids] = clustered_df.shape[0]
    return clustered_df

In [163]:
'''
@param
df: data frame of the responses
id_coord_dict: dictionary with popluated id_coords and the row of the new data frame as the value
num_row: number of rows that the final data frame should be (equal to the cluster rows)
@return
rsp: data frame of the y matrix with the count of species in a plot
'''
def get_responses(df, id_coord_dict,num_row):
    spc = df.spcd.unique().sort()
    zeros = np.zeros(shape=(num_row, len(spc)))
    rsp = pd.DataFrame(0, columns=spc, dtype=int64)
    df['id_coords'] = df.apply(lambda row: str(int(row.statecd)) + '_' + str(int(row.unitcd)) + '_' + str(int(row.countycd)) + '_' + str(int(row[3])), axis=1)
    for index, row in df.itterrows():
        try:
            i = id_coord_dict[row.id_coords]
            rsp.at[i, str(row.spcd)] += row.count
        except:
            pass
    return rsp

In [227]:
Neon_id_coords_dict = {}
cont_id_coords_dict = {}
cols_to_cluster = ["statecd", "unitcd"]
Neon_clusters = cluster_plots(neon_climate, cols_to_cluster, Neon_id_coords_dict)
cont_clusters = cluster_plots(cont_climate, cols_to_cluster, cont_id_coords_dict)

/Users/hyunchoi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyError: 'None of [[  1   5  29  50  72  87  92 114 116 135 140 141 162 163 179 180]] are in the [index]'

In [ ]:
Neon_resp = get_responses(fia_response, Neon_id_coords_dict, Neon_clusters.shape[0])
cont_resp = get_responses(fia_response, cont_clusters, cont_clusters.shape[0])

In [ ]:
Neon_clusters.to_csv(file="Neon_clusters.csv", index=False)
cont_clusters.to_csv(file="cont_clusters.csv", index=False)
Neon_resp.to_csv(file="Neon_clusters.csv", index=False)
cont_resp.to_csv(file="cont_resp.csv",index=False)

In [9]:
neon_climate.to_csv(index=False)
cont_climate.to_csv(index=False)
fia_response.to_csv(index=False)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [5]:
fia_climate

,invyr,statecd,unitcd,countycd,plot,lat,lon,AVG(slope),AVG(aspect),AVG(elev)
0,1998,13,1,1,6,31.514891,-82.193000,0.000000,0.000000,140.0
1,1998,13,1,1,27,31.830661,-82.497255,0.000000,0.000000,240.0
2,1998,13,1,1,35,31.651770,-82.412366,0.000000,0.000000,170.0
3,1998,13,1,1,37,31.550474,-82.154227,0.000000,0.000000,100.0
4,1998,13,1,1,49,31.690276,-82.288088,0.000000,0.000000,190.0
5,1998,13,1,1,52,31.740238,-82.274860,0.000000,0.000000,190.0
6,1998,13,1,1,62,31.645702,-82.348122,0.000000,0.000000,160.0
7,1998,13,1,1,66,31.702446,-82.233569,0.000000,0.000000,180.0
8,1998,13,1,3,3,31.314722,-82.966198,0.000000,0.000000,230.0
9,1998,13,1,3,14,31.305805,-82.861910,0.000000,0.000000,200.0
